In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import time
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("mnist/", one_hot=True)

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


In [2]:
num_train=mnist.train.num_examples
num_val=mnist.validation.num_examples
num_test=mnist.test.num_examples

In [3]:
# parameters
batch_size=64
img_size=28
sensor_unit=256
lstm_size=256
N_glimpse=10
MC_test=64
loc_std=3.0
tot_size=batch_size*MC_test

In [4]:
class Glimpse_Network():
    def __init__(self):
        self.glimspe_size=[5,10,15]
        self.concat_size=5
        self.img_net=tf.layers.Dense(units=sensor_unit,name='glimpse_net/img_net')
        self.loc_net=tf.layers.Dense(units=sensor_unit,name='glimpse_net/loc_net')
        
    def glimpse_sensor(self,image,loc):
        glimpses_list=[tf.image.extract_glimpse(input=image,size=[gs,gs],offsets=loc) for gs in self.glimspe_size]
        glimpses_norm=[tf.image.resize_bilinear(g,[self.concat_size,self.concat_size]) for g in glimpses_list]
        glimpses=tf.concat(values=glimpses_norm,axis=3)  # batch_size*concat_size*concat_size*3
        return glimpses
    
    def forward(self,image,loc):
        glimpses=self.glimpse_sensor(image,loc) # tot_size*concat_size*concat_size*3
        glimpses=tf.stop_gradient(glimpses)  # gradient has no need to flow through glimpses
        g_image=tf.nn.relu(self.img_net(inputs=tf.layers.flatten(glimpses)))
        g_loc=tf.nn.relu(self.loc_net(inputs=loc))
        g_out=tf.nn.relu(g_image+g_loc)
        return g_out

In [5]:
tf.reset_default_graph()
X=tf.placeholder(dtype=tf.float32,shape=[None,28,28,1])
y=tf.placeholder(dtype=tf.int64,shape=[None,10])
this_size=tf.shape(X)[0]
start_location=tf.random_uniform(shape=[this_size,2],minval=-1.0,maxval=1.0)
gNet=Glimpse_Network()

lstm_cell = tf.contrib.rnn.LSTMCell(lstm_size)
state = lstm_cell.zero_state(this_size, tf.float32)

# emission_net_low=tf.layers.Dense(units=128,name='emission_net_low')
emission_net_high=tf.layers.Dense(units=2,name='emission_net_high')
# baseline_net_low=tf.layers.Dense(units=128,name='baseline_net_low')
# baseline_net_mid=tf.layers.Dense(units=128,name='baseline_net_mid')
baseline_net_high=tf.layers.Dense(units=1,name='baseline_net_high')
predict_net=tf.layers.Dense(units=10,name='predict_net')

def loglikelihood(sample,mean):
    gaussian=tf.distributions.Normal(loc=mean,scale=tf.constant([loc_std,loc_std]))
    llh=-gaussian.log_prob(sample)
    return tf.reduce_sum(llh,axis=1)
    
loc_his=[]
loglikelihood_his=[]
baseline_his=[]
normalized_loc=start_location
for ng in range(N_glimpse):
    loc_his.append(normalized_loc)
    
    # extract glimpse
    glimpses_out=gNet.forward(X,normalized_loc)
    
    # RNN
    lstm_output,state=lstm_cell(glimpses_out,state)
    
    # emit mean of location
#     loc_mean=emission_net_high(tf.nn.relu(emission_net_low(lstm_output)))
    loc_mean=emission_net_high(lstm_output)
    
    # sample next location by gaussian distribution centered at loc_mean
    loc_sample=tf.random_normal(shape=(this_size,2),mean=loc_mean,stddev=loc_std)
    loc_sample=tf.stop_gradient(loc_sample)  # very important ***
    
    # calculate the -loglikelihood of the sampled position
    llh=loglikelihood(loc_sample,loc_mean)
    loglikelihood_his.append(llh)
    
    # normalize the location for next input
    normalized_loc=tf.tanh(loc_sample)
    normalized_loc=tf.stop_gradient(normalized_loc)
    
    # output time independent baseline
#     baseline=baseline_net_high(tf.nn.relu(baseline_net_mid(tf.nn.relu(baseline_net_low(lstm_output)))))
    baseline=baseline_net_high(lstm_output)
    baseline_his.append(tf.squeeze(baseline))

# pack data for calculation
baseline_his=tf.stack(baseline_his)
loglikelihood_his=tf.stack(loglikelihood_his)

# make prediction
score=predict_net(inputs=lstm_output)
prediction=tf.argmax(score,1)

# calculate reward, do variance reduction and calculate reinforced loglikelihood
reward=tf.cast(tf.equal(prediction,tf.argmax(y,1)),dtype=tf.float32)
# stop gradient on reward(redundant because tf.equal does not have gradient)
reward=tf.stop_gradient(reward)
accuracy=tf.reduce_sum(reward)/tf.cast(this_size,dtype=tf.float32)
reduce_var_reward=reward-tf.stop_gradient(baseline_his)
reinforce_llh=tf.reduce_mean(loglikelihood_his*reduce_var_reward)

# regression baseline towards reward
baseline_mse=tf.reduce_mean(tf.square(reward-baseline_his))

# softmax to output
softmax_loss=tf.reduce_mean(tf.losses.softmax_cross_entropy(onehot_labels=y,logits=score))

# summarize loss
loss=reinforce_llh+baseline_mse+softmax_loss

# # for testing gradient flow
# dweight1=tf.gradients(loss,[prediction])
# dweight2=tf.gradients(reinforce_llh,[prediction])
# dweight3=tf.gradients(baseline_mse,[prediction])
# dweight4=tf.gradients(softmax_loss,[prediction])
# print(dweight1,dweight2,dweight3,dweight4)


optimizier=tf.train.AdamOptimizer(learning_rate=1e-4)
train_step = optimizier.minimize(loss)

In [6]:
# # for testing gradient flow
# with tf.Session() as sess:
#     tf.global_variables_initializer().run()
#     images,labels=mnist.train.next_batch(1)
#     feed_dict={X:images.reshape(1,28,28,1),y:labels}
#     dw1,dw2,dw3,dw4=sess.run([dweight1,dweight2,dweight3,dweight4],feed_dict=feed_dict)
#     print(dw1[0])
#     print(dw2[0])
#     print(dw3[0])
#     print(dw4[0])

In [7]:
max_epoch=50
print_every=50

def train():
    num_iteration=num_train//batch_size
    for it in range(num_iteration):
        images,labels=mnist.train.next_batch(batch_size)
        # prepare data for monte carlo test
        images=np.tile(images,(MC_test,1))
        labels=np.tile(labels,(MC_test,1))
        feed_dict={X:images.reshape(tot_size,28,28,1),y:labels}
        l1,l2,l3,loss_num,acc_num,_ = sess.run([reinforce_llh,baseline_mse,softmax_loss,loss,accuracy,train_step],feed_dict=feed_dict)
        if it==0 or (it+1)%print_every==0 or it==num_iteration-1:
            print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())),
                  'iteration %d/%d:' % (it+1,num_iteration),'loss=%8f, accuracy=%.3f%%' % (loss_num,acc_num*100.0),l1,l2,l3)

def eval(dataset,num_iteration):
    total_loss=0
    total_accuracy=0
    for it in range(num_iteration):
        images,labels=dataset.next_batch(batch_size)
        # no Monte Carlo test during evaludation step
        loss_num,accuracy_num = sess.run([loss,accuracy],feed_dict={X:images.reshape(batch_size,28,28,1),y:labels})
        total_loss+=loss_num
        total_accuracy+=accuracy_num
    total_loss/=num_iteration
    total_accuracy/=num_iteration
    return total_loss,total_accuracy

In [8]:
acc_train_his=[]
acc_val_his=[]
acc_test_his=[]

saver = tf.train.Saver()

with tf.Session() as sess:
#     print(tf.global_variables())
    tf.global_variables_initializer().run()
    max_acc=None
    for epoch in range(max_epoch):
        print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())),'start epoch %d/%d' % (epoch+1,max_epoch))
        train()
        loss_train,acc_train=eval(mnist.train,500)
        loss_val,acc_val=eval(mnist.validation,70)
        loss_test,acc_test=eval(mnist.test,150)
        acc_train_his.append(acc_train)
        acc_val_his.append(acc_val)
        acc_test_his.append(acc_test)
        
        if max_acc==None or acc_val>max_acc:
            max_acc=acc_val
            save_path = saver.save(sess, "model/RAM/RAM.ckpt")
            print("Currently maximum accuracy on validation set, model saved in path: %s" % save_path)
        
        print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())),'end epoch %d/%d:' % (epoch+1,max_epoch),
             'acc_train=%.3f%% acc_val=%.3f%% acc_test=%.3f%%' % (acc_train*100.0,acc_val*100.0,acc_test*100.0))

2018-04-18 00:34:14 start epoch 1/50
2018-04-18 00:34:15 iteration 1/859: loss=2.803058, accuracy=13.672% 0.37562785 0.1270862 2.300344
2018-04-18 00:34:28 iteration 50/859: loss=2.512510, accuracy=10.840% 0.12480001 0.09821017 2.2895002
2018-04-18 00:34:42 iteration 100/859: loss=2.300372, accuracy=10.547% -0.09995662 0.09584772 2.304481
2018-04-18 00:34:56 iteration 150/859: loss=2.338073, accuracy=12.085% -0.07118176 0.107661426 2.3015935
2018-04-18 00:35:09 iteration 200/859: loss=2.261242, accuracy=9.546% -0.13062114 0.08801276 2.3038507
2018-04-18 00:35:23 iteration 250/859: loss=2.635140, accuracy=15.308% 0.21493849 0.13270104 2.2875006
2018-04-18 00:35:36 iteration 300/859: loss=2.157797, accuracy=12.354% -0.17072386 0.11062133 2.2178993
2018-04-18 00:35:50 iteration 350/859: loss=2.127279, accuracy=15.503% -0.19966449 0.13428777 2.1926556
2018-04-18 00:36:03 iteration 400/859: loss=2.142315, accuracy=14.307% -0.19439119 0.12483909 2.2118673


KeyboardInterrupt: 

In [ ]:
plt.figure(2)
ptr,=plt.plot(range(max_epoch),acc_train_his,'r-')
pva,=plt.plot(range(max_epoch),acc_val_his,'b-')
pte,=plt.plot(range(max_epoch),acc_test_his,'y-')
plt.xlabel('training epoch')
plt.ylabel('accuracy')
plt.title('accuracy on three sets')
plt.legend((ptr,pva,pte),('train','validation','test'))
plt.show()

In [ ]:
saver=tf.train.Saver()
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    saver.restore(sess, "model/RAM/RAM.ckpt")
    _,acc_train=eval(mnist.train,num_train//batch_size)
    _,acc_val=eval(mnist.validation,num_val//batch_size)
    _,acc_test=eval(mnist.test,num_test//batch_size)
    print('Accuracy on training set is %.3f%%' % (acc_train*100.0))
    print('Accuracy on validation set is %.3f%%' % (acc_val*100.0))
    print('Accuracy on testing set is %.3f%%' % (acc_test*100.0))

In [ ]:
# mean=tf.zeros((100,2),dtype=tf.float32)
# std=tf.constant([1,1],dtype=tf.float32)
# gaussian=tf.distributions.Normal(loc=mean,scale=std)
# rand=tf.random_normal(shape=(100,2),mean=0,stddev=1)
# sampled=mean+rand
# prob=-gaussian.log_prob(sampled)
# prob=tf.reduce_mean(tf.reduce_sum(prob,1))
# with tf.Session() as sess:
#     out=sess.run([prob])
#     print(out)